In [ ]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import torch

%load_ext autoreload
%autoreload 2

sns.set()

In [ ]:
parquet_file = 'TCV_LHD_db4ML.parquet.part'
df = pd.read_parquet(parquet_file, engine ='auto')

In [ ]:
print(df.head())
print('----------oooo0000oooo----------')
print(df.info())

In [ ]:
df.describe()

In [ ]:
df['LDH'].values.describe() # values in column LDH

In [ ]:
#df.dropna(inplace=True)
#df.drop(df[df['LDH']  == 'Ip<Ip_MIN'].index, inplace = True)
#df_filter = df[ df['LDH'] != 'Ip<Ip_MIN']

mask = df['LDH'] == 'Ip<Ip_MIN'
df_filter = df.drop(index = df[mask].index) #remove Ip<Ip_MIN values 

df_filter = df_filter.dropna() #remove Nan values
df_filter = df_filter.reset_index(drop=True) #reset indexing
df_filter.LDH = df_filter.LDH.cat.remove_categories('Ip<Ip_MIN') #remove Ip<Ip_MIN category

discard_data = len(df.index) - len(df_filter.index) # number of data point that do not contain useful information
print('number of useless data points: ', discard_data)
print('size of original data set: ', len(df.index))
print('size of filtered data set: ', len(df_filter.index))

print(len(df_filter.index) + discard_data - len(df.index))

print('data statistical description ')
df.describe()

In [ ]:
print('values in LDH column') # to make sure removing actually works
df_filter['LDH'].values.describe() # values in column LDH

In [ ]:
keys = df_filter.keys().to_numpy()
number_wrong_data = np.zeros(len(keys) -1 )
counter = 0
for key in keys[1:]:
    number_wrong_data[counter] = (df_filter[key] == 'Ip<Ip_MIN').mean()
    counter = counter + 1

print(keys[6])

In [ ]:
print(number_wrong_data)
print(keys[1:])
df_filter.head()

In [ ]:
df_filter['LDH'].values
(df_filter['LDH'] == 'Ip<Ip_MIN').mean() # to test Ip<Ip_MIN is actually dropped

In [ ]:
## this plot is really time-consuming

percent_data_plots = 0.01
number_rows = len(df_filter.index)
sample_size = int( number_rows * percent_data_plots )
sample = df_filter.sample(10000) # data sample picked at random
print(sample_size)
print(sample['LDH'].values)
#sns.pairplot(sample, diag_kind='kde',hue = 'LDH', markers=["D", "o", "o", "o"])

In [ ]:
# correlation matrices

corrmat = df_filter.corr()
f, ax = plt.subplots(figsize =(9, 8))
sns.heatmap(corrmat, ax = ax, cmap ="YlGnBu", linewidths = 0.1)
cg = sns.clustermap(corrmat, cmap ="YlGnBu", linewidths = 0.1);
plt.setp(cg.ax_heatmap.yaxis.get_majorticklabels(), rotation = 0)

In [ ]:
# plot of each feature vs labels
ax1 = df_filter.plot.scatter(x='LDH', y='PD', c='DarkBlue')

#plt.plot(df_filter['LDH'], ls = 'none', marker = 'x')

In [ ]:
df_filter.head(20)

In [ ]:
# testing of functions. This can deleted
features = df_filter.keys().to_numpy()
mask_features = features != 'LDH'
features = features[mask_features]

In [ ]:
### PCA computation
features = df_filter.keys().to_numpy()
# ['time' 'IP' 'PD' 'FIR' 'WP' 'LDH' 'pulse']
print(features)
mask = np.array([False, True, True, True, False, False, True ])
print(features[mask])

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from mpl_toolkits.mplot3d import Axes3D

# features 
features = df_filter.keys().to_numpy()
#mask_features_1 = features != 'LDH'
# we remove time, WP, and LDH 
mask_features = np.array([False, True, True, True, False, False, True ])

features = features[mask_features]
print('features in PCA computation', features)

#separate features
x = df_filter.loc[:, features].values
y = df_filter.loc[:,['LDH']].values
x = StandardScaler().fit_transform(x) # Standardizing the features
print(x.shape) # dimension of features
print(y.shape) # dimension of labels

In [ ]:
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data = principalComponents, columns = ['principal component 1', 'principal component 2'])

finalDf = pd.concat([principalDf, df_filter[['LDH']]], axis = 1)

In [ ]:
finalDf.head()

In [ ]:
# PCA visualialization
fig = plt.figure(figsize = (10, 10))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Principal Component 1', fontsize = 12)
ax.set_ylabel('Principal Component 2', fontsize = 12)
ax.set_title('2 component PCA', fontsize = 15)
targets = ['L', 'D', 'H',] # labels to predict
colors = ['r', 'g', 'b',]
for target, color in zip(targets,colors):
    indicesToKeep = finalDf['LDH'] == target
    ax.scatter(finalDf.loc[indicesToKeep, 'principal component 1']
               , finalDf.loc[indicesToKeep, 'principal component 2']
               , c = color
               , s = 50)
ax.legend(targets)
ax.grid()

In [ ]:
# observe values in column WP
df_filter['WP'].describe()

In [ ]:
data_frame = df_filter.copy()
data_frame['LDH'] = np.where(data_frame['LDH'] == 'L', 0)

In [ ]:
### -----------------------------------------------------------
### Analysis of data using time column as describe in the paper:
### Classification of tokamak plasma confinement states with convolutional
### recurrent neural networks

In [ ]:
df_filter.head()

In [ ]:
interval_plot = 1000
plt.plot(df_filter.time[:interval_plot:], df_filter.IP[:interval_plot:], ls = '-', lw = 0.80, c = 'k', label ='IP')
plt.legend()
plt.show()

In [ ]:
# plot of quantities vs time every 100 time steps

interval_plot = 100
plt.plot(df_filter.time[:interval_plot:], df_filter.IP[:interval_plot:], ls = '-', lw = 0.80, c = 'k', label ='IP')
plt.legend()
plt.show()

plt.plot(df_filter.time[:interval_plot:], df_filter.PD[:interval_plot:], ls = '-', lw = 1, c = 'b', label ='PD')
plt.legend()
plt.show()

plt.plot(df_filter.time[:interval_plot:], df_filter.FIR[:interval_plot:], ls = '-', lw = 1, c = 'red', label ='FIR')
plt.legend()
plt.show()

plt.plot(df_filter.time[:interval_plot:], df_filter.WP[:interval_plot:], ls = '-', lw = 1, c = 'green', label ='WP')
plt.legend()
plt.show()

plt.plot(df_filter.time[:interval_plot:], df_filter.pulse[:interval_plot:], ls = '-', lw = 1, c = 'magenta', label ='pulse')
plt.legend()
plt.show()

In [ ]:
# rolling average
step_average = 20

plt.plot(df_filter.IP.rolling(window = step_average).mean(), ls = '-', lw = 0.80, c = 'k', label ='moving mean IP')
plt.legend(loc = 'best')
plt.show()

plt.plot(df_filter.PD.rolling(window = step_average).mean(), ls = '-', lw = 0.80, c = 'b', label ='moving mean PD')
plt.legend(loc = 'best')
plt.show()

plt.plot(df_filter.FIR.rolling(window = step_average).mean(), ls = '-', lw = 0.80, c = 'red', label ='moving mean FIR')
plt.legend(loc = 'best')
plt.show()

plt.plot(df_filter.WP.rolling(window = step_average).mean(), ls = '-', lw = 0.80, c = 'green', label ='moving mean WP')
plt.legend(loc = 'best')
plt.show()

plt.plot(df_filter.pulse.rolling(window = step_average).mean(), ls = '-', lw = 0.80, c = 'magenta', label ='moving mean pulse')
plt.legend(loc = 'best')
plt.show()

In [ ]:
df_filter.IP.rolling(window = step_average).mean().head(30).T

In [ ]:
df_filter.IP.head()